In [75]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [76]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from 'D:\\Anaconda\\Code\\EnSF\\AllenCahn\\AllenCahn_Solver.py'>

In [90]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
T = 10
Nt = 1000
eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

S = 2

cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [91]:
# scipy.io.savemat('RefU0_AllenC_v1.mat', {'U0': U0}) # save initial condition

In [83]:
data = scipy.io.loadmat('RefU0_AllenC_v1.mat')
# data = scipy.io.loadmat('RefU0_AllenC_ConsMob_v1.mat')
# Convert MATLAB arrays to NumPy arrays and cast to float64.
U0 = data['U0'].astype(np.float64)
U0 = U0.reshape(Nx*Ny, )

In [92]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

# Idd = np.eye(N)
# Dh = np.kron(Idd, G)+np.kron(G, Idd)

In [93]:
U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0, N, dt, eps, gamma, S, Dh, cons, noise)

[0.99074421 0.42398698 0.8674228  ... 0.80524688 0.23876128 0.69486911]


In [94]:
RefSol = np.zeros((Nx*Ny, Nt+1))

RefSol[:, 0] += U0
RefSol[:, 1] += U1

In [95]:
for ii in range(1, Nt):
    print(f'step={ii}:')
    t1 = time.time() 
    x = AC_Neu.AllenCahn_Solver_1step_BDF2(U1, U0, N, dt, eps, gamma, S, Dh, b0, b1, cons)
    t2 = time.time()
    print(f'\t time = {t2 - t1:.4f}')
    
    U0 = U1.copy()
    U1 = x.copy()
    RefSol[:, ii+1] +=x

step=1:
[0.99479567 0.45079554 0.88700222 ... 0.81239655 0.25429721 0.71258789]
	 time = 0.2777
step=2:
[0.99767156 0.47835248 0.90518694 ... 0.81952578 0.27105591 0.73036068]
	 time = 0.2504
step=3:
[0.99936758 0.50557072 0.92146311 ... 0.82651477 0.28862962 0.74779528]
	 time = 0.2491
step=4:
[0.99999088 0.53201211 0.93577251 ... 0.83332307 0.30686014 0.76474022]
	 time = 0.2533
step=5:
[0.99967325 0.55746035 0.94820194 ... 0.8399365  0.32566043 0.78112313]
	 time = 0.2457
step=6:
[0.99854576 0.58178897 0.95888457 ... 0.84635016 0.34496048 0.79690018]
	 time = 0.2357
step=7:
[0.99673144 0.60491906 0.96796706 ... 0.8525629  0.36469114 0.81204041]
	 time = 0.2347
step=8:
[0.994343   0.62680419 0.97559733 ... 0.85857552 0.38478003 0.82652073]
	 time = 0.2298
step=9:
[0.9914823  0.64742342 0.98191925 ... 0.86439007 0.40515096 0.84032442]
	 time = 0.2392
step=10:
[0.98824039 0.66677671 0.98706985 ... 0.87000957 0.42572479 0.85344041]
	 time = 0.2272
step=11:
[0.9846979  0.68488111 0.99117

	 time = 0.2068
step=87:
[0.84386264 0.87941206 0.9133832  ... 0.99906371 0.9858933  0.98237845]
	 time = 0.2056
step=88:
[0.84327505 0.87889519 0.91297033 ... 0.99885269 0.98648797 0.98214489]
	 time = 0.2109
step=89:
[0.84269195 0.87837528 0.91257042 ... 0.99862247 0.9870598  0.98192503]
	 time = 0.2046
step=90:
[0.84211304 0.87785271 0.91218305 ... 0.99837358 0.98760994 0.98171877]
	 time = 0.2104
step=91:
[0.84153802 0.87732784 0.91180782 ... 0.99810651 0.98813948 0.98152599]
	 time = 0.2100
step=92:
[0.84096665 0.87680101 0.91144437 ... 0.99782176 0.98864944 0.98134655]
	 time = 0.2115
step=93:
[0.84039866 0.87627253 0.91109231 ... 0.9975198  0.98914076 0.98118032]
	 time = 0.1861
step=94:
[0.83983383 0.87574268 0.91075131 ... 0.99720105 0.98961433 0.98102715]
	 time = 0.2061
step=95:
[0.83927193 0.87521176 0.91042104 ... 0.99686596 0.99007098 0.98088688]
	 time = 0.2105
step=96:
[0.83871276 0.87468002 0.91010117 ... 0.99651491 0.99051148 0.98075937]
	 time = 0.2430
step=97:
[0.83

	 time = 0.2121
step=171:
[0.7997827  0.8408206  0.90481595 ... 0.93692513 0.99717454 0.99398056]
	 time = 0.2057
step=172:
[0.79930715 0.84050338 0.90494194 ... 0.93571555 0.99692932 0.994265  ]
	 time = 0.2243
step=173:
[0.79883367 0.84019068 0.90507299 ... 0.93449399 0.9966724  0.99454633]
	 time = 0.2209
step=174:
[0.79836231 0.83988253 0.9052091  ... 0.93326039 0.99640359 0.99482427]
	 time = 0.2171
step=175:
[0.79789312 0.83957898 0.90535031 ... 0.93201469 0.9961227  0.99509853]
	 time = 0.2108
step=176:
[0.79742615 0.83928006 0.90549662 ... 0.93075682 0.99582952 0.99536881]
	 time = 0.2167
step=177:
[0.79696146 0.83898582 0.90564805 ... 0.92948671 0.99552387 0.99563482]
	 time = 0.2172
step=178:
[0.79649911 0.83869629 0.90580463 ... 0.9282043  0.99520554 0.99589625]
	 time = 0.2111
step=179:
[0.79603913 0.83841152 0.90596637 ... 0.92690953 0.99487434 0.99615279]
	 time = 0.2178
step=180:
[0.79558159 0.83813155 0.90613329 ... 0.92560233 0.99453004 0.99640414]
	 time = 0.2064
step

	 time = 0.2378
step=256:
[0.77245262 0.83448209 0.93474569 ... 0.78726972 0.91339704 0.97091586]
	 time = 0.2230
step=257:
[0.77236659 0.83471654 0.93532539 ... 0.78494751 0.91141463 0.96956979]
	 time = 0.2214
step=258:
[0.77228806 0.83495964 0.93590957 ... 0.78261406 0.90940558 0.96818741]
	 time = 0.2237
step=259:
[0.77221712 0.83521145 0.93649817 ... 0.78026951 0.90736991 0.96676848]
	 time = 0.2185
step=260:
[0.77215382 0.835472   0.93709111 ... 0.777914   0.90530766 0.96531278]
	 time = 0.2314
step=261:
[0.77209826 0.83574133 0.93768833 ... 0.77554766 0.90321889 0.96382008]
	 time = 0.2276
step=262:
[0.77205051 0.8360195  0.93828977 ... 0.77317066 0.90110365 0.9622902 ]
	 time = 0.2211
step=263:
[0.77201064 0.83630656 0.93889534 ... 0.77078313 0.89896202 0.96072296]
	 time = 0.2174
step=264:
[0.77197873 0.83660253 0.93950498 ... 0.76838523 0.89679406 0.95911818]
	 time = 0.2123
step=265:
[0.77195486 0.83690747 0.94011861 ... 0.76597713 0.89459988 0.95747573]
	 time = 0.2228
step

	 time = 0.2414
step=340:
[0.7988735  0.88711672 0.98869491 ... 0.5711703  0.67557713 0.74192413]
	 time = 0.2427
step=341:
[0.79969077 0.88814609 0.98921362 ... 0.56860117 0.67232854 0.73830736]
	 time = 0.2561
step=342:
[0.80052189 0.88918374 0.98972349 ... 0.56603822 0.66908298 0.73468807]
	 time = 0.2416
step=343:
[0.8013669  0.89022958 0.99022428 ... 0.56348164 0.66584084 0.73106693]
	 time = 0.2546
step=344:
[0.80222586 0.89128352 0.99071574 ... 0.56093158 0.66260255 0.72744457]
	 time = 0.2490
step=345:
[0.80309881 0.89234542 0.9911976  ... 0.55838823 0.65936848 0.72382164]
	 time = 0.2063
step=346:
[0.8039858  0.89341519 0.99166963 ... 0.55585173 0.65613903 0.72019877]
	 time = 0.2579
step=347:
[0.80488687 0.89449271 0.99213155 ... 0.55332226 0.65291459 0.71657657]
	 time = 0.2708
step=348:
[0.80580208 0.89557785 0.99258311 ... 0.55079997 0.64969553 0.71295566]
	 time = 0.2680
step=349:
[0.80673145 0.8966705  0.99302405 ... 0.548285   0.64648221 0.70933663]
	 time = 0.2372
step

...

	 time = 0.3545
step=592:
[0.68377799 0.61784401 0.50955007 ... 0.20398634 0.21609788 0.22345586]
	 time = 0.3489
step=593:
[0.67937644 0.61400203 0.50667682 ... 0.20335209 0.21537164 0.2226738 ]
	 time = 0.2016
step=594:
[0.6749929  0.61017987 0.50382038 ... 0.20272128 0.21464967 0.22189654]
	 time = 0.3749
step=595:
[0.67062809 0.60637789 0.50098083 ... 0.20209389 0.21393194 0.22112404]
	 time = 0.3711
step=596:
[0.66628269 0.60259642 0.49815825 ... 0.20146989 0.21321842 0.22035626]
	 time = 0.3453
step=597:
[0.66195733 0.59883579 0.49535269 ... 0.20084927 0.21250908 0.21959317]
	 time = 0.3575
step=598:
[0.65765265 0.59509629 0.49256423 ... 0.20023199 0.21180388 0.21883472]
	 time = 0.3573
step=599:
[0.65336921 0.59137822 0.48979293 ... 0.19961804 0.21110278 0.21808088]
	 time = 0.3693
step=600:
[0.64910759 0.58768184 0.48703882 ... 0.1990074  0.21040577 0.2173316 ]
	 time = 0.3486
step=601:
[0.64486831 0.5840074  0.48430195 ... 0.19840002 0.2097128  0.21658686]
	 time = 0.3678
step

	 time = 0.3899
step=676:
[0.40196038 0.37373974 0.32575137 ... 0.1607173  0.16731583 0.17137334]
	 time = 0.3566
step=677:
[0.39966132 0.37172358 0.32418479 ... 0.16030301 0.16685622 0.17088702]
	 time = 0.4085
step=678:
[0.39738274 0.36972447 0.32263014 ... 0.15989067 0.1663989  0.1704032 ]
	 time = 0.3564
step=679:
[0.39512444 0.36774227 0.32108731 ... 0.15948025 0.16594387 0.16992188]
	 time = 0.3960
step=680:
[0.39288623 0.36577682 0.31955621 ... 0.15907176 0.16549109 0.16944304]
	 time = 0.3691
step=681:
[0.39066792 0.36382796 0.31803673 ... 0.15866517 0.16504057 0.16896665]
	 time = 0.3470
step=682:
[0.38846932 0.36189554 0.31652878 ... 0.15826047 0.16459228 0.1684927 ]
	 time = 0.3977
step=683:
[0.38629023 0.35997941 0.31503227 ... 0.15785765 0.1641462  0.16802117]
	 time = 0.3979
step=684:
[0.38413047 0.35807941 0.31354709 ... 0.15745671 0.16370233 0.16755205]
	 time = 0.3978
step=685:
[0.38198984 0.3561954  0.31207315 ... 0.15705762 0.16326064 0.16708532]
	 time = 0.4215
step

	 time = 0.4378
step=760:
[0.26403193 0.25085545 0.22737706 ... 0.13166539 0.13542301 0.13782343]
	 time = 0.3914
step=761:
[0.26289436 0.24982309 0.22652199 ... 0.13137866 0.1351116  0.13749781]
	 time = 0.4337
step=762:
[0.26176542 0.24879822 0.22567259 ... 0.13109309 0.13480153 0.13717362]
	 time = 0.4367
step=763:
[0.26064502 0.24778076 0.2248288  ... 0.13080869 0.13449277 0.13685085]
	 time = 0.4328
step=764:
[0.25953307 0.24677065 0.22399059 ... 0.13052543 0.13418534 0.1365295 ]
	 time = 0.4349
step=765:
[0.2584295  0.24576781 0.22315789 ... 0.13024333 0.1338792  0.13620954]
	 time = 0.4397
step=766:
[0.25733421 0.24477217 0.22233066 ... 0.12996236 0.13357437 0.13589098]
	 time = 0.4359
step=767:
[0.25624712 0.24378366 0.22150885 ... 0.12968253 0.13327083 0.13557381]
	 time = 0.4379
step=768:
[0.25516816 0.24280221 0.22069242 ... 0.12940382 0.13296857 0.13525801]
	 time = 0.4374
step=769:
[0.25409724 0.24182777 0.21988132 ... 0.12912624 0.13266759 0.13494358]
	 time = 0.4477
step

	 time = 0.4789
step=844:
[0.1918957  0.18466276 0.17139509 ... 0.11109458 0.11324239 0.11472515]
	 time = 0.4702
step=845:
[0.19125646 0.18406923 0.17088187 ... 0.11088656 0.11301971 0.11449422]
	 time = 0.4840
step=846:
[0.19062114 0.18347921 0.17037148 ... 0.11067928 0.11279785 0.11426417]
	 time = 0.5300
step=847:
[0.1899897  0.18289267 0.16986389 ... 0.11047274 0.11257681 0.11403499]
	 time = 0.4952
step=848:
[0.18936212 0.18230958 0.16935909 ... 0.11026694 0.11235659 0.11380667]
	 time = 0.4553
step=849:
[0.18873835 0.18172992 0.16885705 ... 0.11006186 0.11213719 0.11357922]
	 time = 0.4734
step=850:
[0.18811837 0.18115366 0.16835776 ... 0.10985752 0.11191859 0.11335262]
	 time = 0.4600
step=851:
[0.18750215 0.18058077 0.16786118 ... 0.1096539  0.1117008  0.11312687]
	 time = 0.4872
step=852:
[0.18688965 0.18001122 0.16736731 ... 0.109451   0.11148381 0.11290198]
	 time = 0.4773
step=853:
[0.18628085 0.17944498 0.16687611 ... 0.10924881 0.11126761 0.11267792]
	 time = 0.4645
step

	 time = 0.4995
step=928:
[0.14923627 0.14476439 0.13641153 ... 0.09589525 0.09705083 0.09798328]
	 time = 0.5002
step=929:
[0.14883627 0.14438743 0.13607617 ... 0.09573857 0.09688471 0.09781203]
	 time = 0.5152
step=930:
[0.14843829 0.14401231 0.13574236 ... 0.09558239 0.09671914 0.09764135]
	 time = 0.4931
step=931:
[0.14804231 0.14363903 0.13541009 ... 0.0954267  0.09655411 0.09747124]
	 time = 0.5399
step=932:
[0.14764832 0.14326757 0.13507936 ... 0.09527151 0.09638962 0.09730169]
	 time = 0.4364
step=933:
[0.14725631 0.14289792 0.13475014 ... 0.09511681 0.09622566 0.0971327 ]
	 time = 0.5557
step=934:
[0.14686625 0.14253006 0.13442243 ... 0.0949626  0.09606223 0.09696427]
	 time = 0.6187
step=935:
[0.14647814 0.14216399 0.13409623 ... 0.09480887 0.09589933 0.0967964 ]
	 time = 0.5006
step=936:
[0.14609196 0.1417997  0.13377152 ... 0.09465563 0.09573697 0.09662908]
	 time = 0.5028
step=937:
[0.14570771 0.14143716 0.13344828 ... 0.09450287 0.09557512 0.09646231]
	 time = 0.4324
step

In [96]:
scipy.io.savemat('TestRefSol_AllenC_T10_v1.mat', {'RefU':RefSol})